In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
import re
import requests
import json
from bs4 import BeautifulSoup
import time

# Merge all Usernames into one file

In [48]:
#usernames_boho.txt, usernames_hc.txt, usernames_wl.txt
boho = pd.read_csv("usernames_boho.txt", names = ['username'])
HC = pd.read_csv("usernames_hc.txt", names = ['username'])
WL = pd.read_csv("usernames_wl.txt", names = ['username'])

usernames = (pd.concat([WL, HC, boho])).reset_index()

In [49]:
for i,v in enumerate(usernames.username):
    usernames.username[i] = usernames.username[i][0:-4]


/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Then scrape the usernames' # followers from IG

 Markdown
path = '/Users/kimia/Desktop/' #Capstone/EMPORIOSIRENUSE_20173012/zeusndione/followers data
exDF = pd.read_csv(path+"dodobaror_followers_posts.csv")
usernames = exDF.username.unique()
len(usernames.username)

In [54]:
f = usernames.username
follower_count = []
users = []
for i, v in enumerate(f):
    
    username = f[i]
    #print("initial" , username)
    response = requests.get('https://www.instagram.com/' + str(username))
    #print("response" )
    page = response.content
    #print("page")
    soup = BeautifulSoup(page, 'html.parser')
    #print("soup")
    #if line = re.search("(.*?)\sFollowers", str(soup)).group(1)
    #number_followers = re.search("(\d+)\sFollowers", str(soup)).group(1) ## this doesnt work for 12.3k 
    try: 
        line = re.search("(.*?)\sFollowers", str(soup)).group(1)
        number = line.rsplit('\"')[-1] 
    
        #print("#fol")
        
        follower_count.append(number)
        users.append(username)
        time.sleep(20)
    except: 
        print("404: user may have deleted account, gone to private, etc")
        

In [55]:
result = pd.DataFrame({'username':users, 'followers': follower_count})
result.to_csv('users_followers_4.29.csv')

_______________________________________________________________________

## get likes and comment count per pic

In [148]:

path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/'

dfs = []
for i,v in enumerate(result.username):
    jsonfile = str(v) + '.json'
    file = path+"User_pics/All/"  + str(jsonfile)
    with open(file) as train_file:
        dict_train = json.load(train_file)
        df = pd.DataFrame.from_dict(json_normalize(dict_train), orient='columns')
        dfs.append(df)
        
df_all_meta = (pd.concat(dfs)).reset_index()
df_all_meta.username = df_all_meta.owner_username
"""df_all_meta.to_csv('df_all_meta.csv')"""


In [158]:
df_all_meta['username'] = df_all_meta['owner_username']
df_all_meta.columns


Index(['index', 'caption', 'caption_hashtags', 'caption_mentions', 'comments',
       'date_local', 'date_utc', 'is_video', 'likes', 'mediaid', 'owner_id',
       'owner_username', 'shortcode', 'tagged_users', 'typename', 'url',
       'video_url', 'username'],
      dtype='object')

In [159]:
#df_all_meta.head()
DF_metadata= pd.merge(df_all_meta, result, on='username',)

In [161]:
DF_metadata.to_csv('DF_metadata_COMPLETE.csv')

In [104]:
kkkk = list(train['index'])

In [72]:
path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/'
df = pd.read_csv(path+"users_followers.csv")

In [131]:
#df.followers = df.followers.replace("k","000").replace("m","000000").replace(",","").replace(".","")
df.followers = pd.to_numeric(df.followers)

# Change k and m to numbers, turn columns from str --> int

In [133]:
df['followers'] = (df['followers'].apply(lambda x: x.replace('k',"000").replace(".","").replace("m","000000").replace(",","")))
df.followers = pd.to_numeric(df.followers)

In [135]:
df = result.merge(exDF.drop_duplicates('username'),how='left',on='username')

In [136]:
df.head()

,followers,username,id_post,video_count,url_img,link_post,owner,caption,comment_count,taken_at_timestamp,taken_at_time,shortcode,is_video,likes_count
0,1283,or_shmul,1627655237685711385,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/BaWl9CjD5IZ,45547257,💞,7,1508251646,2017-10-17 14:47:26,BaWl9CjD5IZ,False,138
1,741,fathy_abo_dahis,1641992705372560728,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/BbJh6p3g51Y,3470341263,NaN,1,1509960805,2017-11-06 09:33:25,BbJh6p3g51Y,False,24
2,867,iamkurdapya,1644299880334917484,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/BbRugd_lH9s,226907530,☝,0,1510235842,2017-11-09 13:57:22,BbRugd_lH9s,False,6
3,5131,abhishekraghav015,1637736956104215547,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/Ba6aRWgHwv7,3106737762,#like4like#followforfollow#justinbieber#selena...,9,1509453480,2017-10-31 12:38:00,Ba6aRWgHwv7,False,211
4,5051,misstribu,1643430709473145791,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/BbOo4XjHXO_,595372894,#regram @nathalie_dmitrovic #newshop #marseill...,2,1510132229,2017-11-08 09:10:29,BbOo4XjHXO_,False,55


In [138]:
df.to_csv('users_followers_merged.csv')

In [156]:
import subprocess
subprocess.check_output(["instaloader", "jnbax"])


b'Stored ID 204834620 for profile jnbax.\njnbax/2015-11-03_00-03-01_UTC_profile_pic.jpg \nRetrieving posts from profile jnbax.\n[  1/  3] jnbax/2017-09-23_16-45-09_UTC.jpg <no caption> \n[  2/  3] jnbax/2017-09-23_03-06-23_UTC.jpg <no caption> \n[  3/  3] jnbax/2013-07-28_04-41-18_UTC.jpg <no caption> \n'

In [148]:
list(df.username)

['or_shmul',
 'fathy_abo_dahis',
 'iamkurdapya',
 'abhishekraghav015',
 'misstribu',
 'moranmarima',
 'elizabethvdg',
 'accessories_phone_sale',
 'clairesulmers',
 'adi_greenberger',
 'baretrogi',
 '_sofiadangelo_',
 'kingkaitlyn05',
 'patriciaortiz__design',
 'havalava',
 'paolanmd',
 'giorgiacantarini',
 'desiree.dibella',
 'fashionandstyle5102',
 'clarespieler',
 'rotembentora2327',
 'yaelshachar12',
 'zuricelis',
 'lnfitz0203',
 'pazit82',
 'netayam',
 'natalil3',
 'lironwiner',
 'sjnippiesworld',
 'mariagraziadeluca',
 'gali_deck',
 'honigman_kids_bet_shean',
 'samheck',
 'beauty_fashion9436',
 'roni_tesler',
 'avital_mat',
 'morin_atar',
 'tali_berkovichmanes',
 'jungxah',
 'rinatreuvenmaya',
 'idan_hasson',
 'nizahajaj',
 'hadarcad',
 'claudia.dellapace',
 'emstyled']

___________________________________________________

## analysis

In [162]:

path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/'
df = pd.read_csv(path+'users_followers_merged.csv')

In [167]:
def zscore(x,mu,std):
    zscore = (x-mu)/std
    return(zscore)

def metrics(df): 
    df[['likes_count','followers']] = df[['likes_count','followers']].apply(pd.to_numeric)

    df['likes_score'] = (df['likes_count'])/(df['followers'])
    
    df['comments_score'] = (df['comment_count'])/(df['followers'])
    
    df['zscore_likes'] = zscore(df['likes_score'], df.likes_score.std(), df.likes_score.mean())
    
    df['zscore_comments'] = zscore(df['comments_score'], df.comments_score.std(), df.comments_score.mean())
    
    df['final_score'] = df['zscore_likes'] + df['zscore_comments']
    

In [169]:
metrics(df)

## integrate with KL for final combination

In [140]:
metrics(df)

In [113]:
## fake KL score for testing

from random import randint


df['KLscore'] = 0
for i in enumerate(df['KLscore']):
    df['KLscore'][i] = randint(0, 9)
#print(randint(0, 9))

/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [122]:
df.head()

,followers,username,id_post,video_count,url_img,link_post,owner,caption,comment_count,taken_at_timestamp,taken_at_time,shortcode,is_video,likes_count,likes_score,zscore,KLscore,followers_score
0,188,abhishekraghav015,1637736956104215547,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/Ba6aRWgHwv7,3106737762,#like4like#followforfollow#justinbieber#selena...,9,1509453480,2017-10-31 12:38:00,Ba6aRWgHwv7,False,211,1.122340,2.506141,5,3.007369
1,47,misstribu,1643430709473145791,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/BbOo4XjHXO_,595372894,#regram @nathalie_dmitrovic #newshop #marseill...,2,1510132229,2017-11-08 09:10:29,BbOo4XjHXO_,False,55,1.170213,2.663942,9,5.754115
2,464,moranmarima,1644371575393638121,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/BbR-zxOFlbp,255202154,@anatfriedman FW_ 17/18\nPH | @liorandhadas \n...,0,1510244388,2017-11-09 16:19:48,BbR-zxOFlbp,False,24,0.051724,-1.022920,7,-1.718506
3,862,elizabethvdg,1642287665808460011,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/BbKk-5LhMjr,1495458765,Thank you @thezoereport #nothinglikehighshine ...,41,1509995967,2017-11-06 19:19:27,BbKk-5LhMjr,False,442,0.512761,0.496791,2,0.238460
4,457,accessories_phone_sale,1641509454729825470,107,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/BbH0Cboncy-,6046227131,עוד לקוחה מרוצה ✔️ @shirmuharer,1,1509903225,2017-11-05 17:33:45,BbH0Cboncy-,True,44,0.096280,-0.876051,0,-0.000000


In [128]:
df['final_followers_score'] = .4*(df['zscore']) * (.6*(df['KLscore']) )

## Here we have a linear combination

In [131]:
col = ['KLscore','final_followers_score', 'zscore']
sorted1 = df[col].sort_values(['final_followers_score'],ascending=False)
sorted1

,KLscore,final_followers_score,zscore
1,9,5.754115,2.663942
0,5,3.007369,2.506141
3,2,0.238460,0.496791
4,0,-0.000000,-0.876051
13,0,-0.000000,-0.985400
14,5,-0.199720,-0.166433
6,3,-0.302045,-0.419506
5,2,-0.329789,-0.687060
7,4,-0.698937,-0.728059
10,7,-0.704771,-0.419506


# score = z + KL^alpha

In [137]:
alpha = 2
df['final_followers_score'] = (df['zscore']) * ((df['KLscore']) * alpha)
col = ['KLscore','final_followers_score', 'zscore']
sorted1 = df[col].sort_values(['final_followers_score'],ascending=False)
sorted1

,KLscore,final_followers_score,zscore
1,9,47.950962,2.663942
0,5,25.061412,2.506141
3,2,1.987164,0.496791
4,0,-0.000000,-0.876051
13,0,-0.000000,-0.985400
14,5,-1.664332,-0.166433
6,3,-2.517039,-0.419506
5,2,-2.748241,-0.687060
7,4,-5.824475,-0.728059
10,7,-5.873090,-0.419506


# score = ln(z) + alpha*ln(k)

#### this has problems. Need to take abs value and some go to -inf

In [135]:
alpha = 2
df['final_followers_score'] = np.log(np.abs(df['zscore'])) + (alpha* np.log((df['KLscore']) ))

col = ['KLscore','final_followers_score', 'zscore']
sorted1 = df[col].sort_values(['final_followers_score'],ascending=False)
sorted1

/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


,KLscore,final_followers_score,zscore
1,9,5.374256,2.663942
0,5,4.137620,2.506141
9,9,4.019116,-0.687060
2,7,3.914482,-1.022920
10,7,3.023144,-0.419506
11,5,2.901503,-0.728059
8,4,2.693593,-0.924044
12,4,2.693593,-0.924044
7,4,2.455216,-0.728059
14,5,1.425715,-0.166433
